# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gamboma,-1.8764,15.8644,23.25,64,76,2.33,CG,1722912760
1,1,barabai,-2.5833,115.3833,32.85,49,38,0.90,ID,1722912907
2,2,adamstown,-25.0660,-130.1015,19.20,69,43,6.14,PN,1722912908
3,3,hadibu,12.6500,54.0333,25.17,84,39,8.30,YE,1722912634
4,4,waitangi,-43.9535,-176.5597,11.57,79,9,2.24,NZ,1722912911


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cond_df = city_data_df.loc[((city_data_df["Max Temp"] <= 27.0) & (city_data_df["Max Temp"] >= 21.0))\
                                 & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
ideal_weather_cond_df = ideal_weather_cond_df.dropna()

# Display sample data
ideal_weather_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
111,111,arraial do cabo,-22.9661,-42.0278,22.45,87,0,11.10,BR,1722913032
164,164,ar rutbah,33.0381,40.2848,25.61,39,0,5.17,IQ,1722913093
201,201,mount isa,-20.7333,139.5000,23.87,41,0,4.12,AU,1722913136
203,203,aquidauana,-20.4711,-55.7872,24.10,34,0,2.18,BR,1722913138
230,230,larache,35.1932,-6.1557,21.06,93,0,3.32,MA,1722913168


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cond_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
111,arraial do cabo,BR,-22.9661,-42.0278,87,
164,ar rutbah,IQ,33.0381,40.2848,39,
201,mount isa,AU,-20.7333,139.5000,41,
203,aquidauana,BR,-20.4711,-55.7872,34,
230,larache,MA,35.1932,-6.1557,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [61]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "apiKey":geoapify_key,
        "limit": 1,  
        "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arraial do cabo - nearest hotel: No hotel found
ar rutbah - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
aquidauana - nearest hotel: Hotel Pantaneiro
larache - nearest hotel: Hotel Tanger
rio verde de mato grosso - nearest hotel: Balneário e Pousada Paraíso
derbent - nearest hotel: Отель Вояж
morehead - nearest hotel: No hotel found
morganton - nearest hotel: Fairfield Inn & Suites Morganton Historic Downtown
remire-montjoly - nearest hotel: Complexe Belova
ilhabela - nearest hotel: Vila Kebaya
ketchikan - nearest hotel: Cape Fox Lodge
sirte - nearest hotel: فندق المدينة-سرت
argo - nearest hotel: Apollon Hotel
kangding - nearest hotel: 溜溜城格桑花大酒店
grottaglie - nearest hotel: Monun Hotel Restaurant & Spa
sayat - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
elkins - nearest hotel: Holiday Inn Express Inn and Suites
itacoatiara - nearest hotel: Regine's Hotel
kebili - nearest hotel: Hôtel du Autruches
sakakah - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
111,arraial do cabo,BR,-22.9661,-42.0278,87,No hotel found
164,ar rutbah,IQ,33.0381,40.2848,39,No hotel found
201,mount isa,AU,-20.7333,139.5000,41,Ibis Styles
203,aquidauana,BR,-20.4711,-55.7872,34,Hotel Pantaneiro
230,larache,MA,35.1932,-6.1557,93,Hotel Tanger
237,rio verde de mato grosso,BR,-18.9181,-54.8442,31,Balneário e Pousada Paraíso
240,derbent,RU,42.0678,48.2899,76,Отель Вояж
285,morehead,US,37.2711,-87.1764,95,No hotel found
320,morganton,US,35.7454,-81.6848,90,Fairfield Inn & Suites Morganton Historic Down...
351,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [101]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriImagery",
    frame_width=700,
    frame_height=625,
    scale=0.5,
    color="City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)